<a href="https://colab.research.google.com/github/tjwei/taishin20190827/blob/master/CF/knn_txn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 用 KNN 的方式做推薦

台新分析開發環境案客戶資料
https://drive.google.com/drive/u/2/folders/1AAXWPcB9xiDe_qX87ce5XL-utkASWxRe

請先連到這個分享資料夾，並且按下右鍵，加入你的 Google 雲端硬碟


In [1]:
# 這個是授權存取你的 Google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 確認真的連上了資料
!ls drive/My\ Drive/台新分析開發環境案客戶資料

'Copy of 0606_口述台新資料欄位.m4a'
'Copy of 0610_嘉駿寄來的欄位內容.png'
'Copy of 0621_data_original.zip'
'Copy of profile.csv'
'Copy of profile.gsheet'
'Copy of result_y1.csv'
'Copy of result_y1.gsheet'
'Copy of result_y2.csv'
'Copy of sr_1.csv'
'Copy of sr_2.csv'
'Copy of sr_3.csv'
'Copy of sr_4.csv'
'Copy of status.csv'
'Copy of visa-merchant-data-standards-manual_用來對照sr_1.csv中的mcc欄位.pdf'
'Copy of zip 對照表.xlsx'
'Copy of 檔案欄位說明_MOOCS.XLSX'


In [0]:
# 正式開始
import pandas as pd
import os.path
import numpy as np

In [0]:
DATA_DIR='drive/My Drive/台新分析開發環境案客戶資料'
sr_1_fn = os.path.join(DATA_DIR, 'Copy of sr_1.csv')
with open(sr_1_fn, encoding='cp950', errors='replace') as f:
    df = pd.read_csv(f, nrows=1000000, usecols=['CUST_NO', 'TXN_DESC'])

In [5]:
df.head()

,CUST_NO,TXN_DESC
0,564755,全聯台中四平分公司 TAICHU/TW
1,564755,全聯台中四平分公司 TAICHU/TW
2,6185152,健身工廠九如廠 Kaohsi/TW
3,6185152,富邦ｍｏｍｏ－ＥＣ TAIPEI/TW
4,5318034,國泰人壽保費部 第 12/12 期 /TW


In [6]:
df.TXN_DESC.nunique()

204796

In [7]:
TXN = df.TXN_DESC.str.extract(r'([^\s０－９（５１２）]+)')
df['TXN_DESC'] = TXN
df.TXN_DESC.nunique()

41704

In [8]:
# 看一下使用頻率
txn_counts = df.TXN_DESC.value_counts()
txn_counts.quantile(np.arange(.5,1,.01))

0.50      2.0
0.51      2.0
0.52      2.0
0.53      2.0
0.54      2.0
0.55      2.0
0.56      2.0
0.57      2.0
0.58      2.0
0.59      2.0
0.60      2.0
0.61      2.0
0.62      2.0
0.63      3.0
0.64      3.0
0.65      3.0
0.66      3.0
0.67      3.0
0.68      3.0
0.69      3.0
0.70      3.0
0.71      4.0
0.72      4.0
0.73      4.0
0.74      4.0
0.75      5.0
0.76      5.0
0.77      5.0
0.78      6.0
0.79      6.0
0.80      6.0
0.81      7.0
0.82      7.0
0.83      8.0
0.84      9.0
0.85     10.0
0.86     11.0
0.87     13.0
0.88     14.0
0.89     16.0
0.90     19.0
0.91     22.0
0.92     26.0
0.93     31.0
0.94     38.0
0.95     48.0
0.96     63.0
0.97     84.0
0.98    118.0
0.99    218.0
Name: TXN_DESC, dtype: float64

In [9]:
# 只留下交易一定次數以上的商店
good_txn = txn_counts[txn_counts>50]
#df = df.loc[df.TXN_DESC.isin(good_txn.index)]
df.drop(df.loc[~df.TXN_DESC.isin(good_txn.index)].index, inplace=True)
df.TXN_DESC.nunique()

2016

In [10]:
# 看看用戶交易次數
cust_counts = df.CUST_NO.value_counts()
cust_counts.quantile(np.arange(0,1,.01))

0.00     1.0
0.01     1.0
0.02     1.0
0.03     1.0
0.04     1.0
0.05     1.0
0.06     1.0
0.07     1.0
0.08     1.0
0.09     1.0
0.10     1.0
0.11     1.0
0.12     1.0
0.13     1.0
0.14     1.0
0.15     1.0
0.16     1.0
0.17     1.0
0.18     1.0
0.19     1.0
0.20     2.0
0.21     2.0
0.22     2.0
0.23     2.0
0.24     2.0
0.25     2.0
0.26     2.0
0.27     2.0
0.28     2.0
0.29     2.0
        ... 
0.70     7.0
0.71     7.0
0.72     8.0
0.73     8.0
0.74     8.0
0.75     8.0
0.76     9.0
0.77     9.0
0.78     9.0
0.79    10.0
0.80    10.0
0.81    10.0
0.82    11.0
0.83    11.0
0.84    11.0
0.85    12.0
0.86    12.0
0.87    13.0
0.88    14.0
0.89    14.0
0.90    15.0
0.91    16.0
0.92    17.0
0.93    18.0
0.94    19.0
0.95    21.0
0.96    23.0
0.97    25.0
0.98    29.0
0.99    37.0
Name: CUST_NO, Length: 100, dtype: float64

In [11]:
good_cust = cust_counts[cust_counts>15]
#df = df.loc[df.CUST_NO.isin(good_cust.index)]
df.drop(df.loc[~df.CUST_NO.isin(good_cust.index)].index, inplace=True)
print(df.TXN_DESC.nunique())
print(df.CUST_NO.nunique())

2016
11620


In [12]:
# pivot dataframe, 變成我們要的矩陣
df = df.pivot_table(index='TXN_DESC', columns='CUST_NO',  aggfunc=len, fill_value=0 )
df

CUST_NO,44429,46973,48428,49499,50758,51460,53517,54748,56868,58179,59378,60437,61599,61714,62122,63941,65314,65822,65859,67296,67362,70605,71301,72538,73073,75468,76602,76635,77411,77746,79937,79947,82362,82606,83272,84636,86344,87891,89253,89318,...,11005829,11007055,11007160,11008686,11009806,11011328,11011845,11012714,11012984,11014260,11016498,11019153,11019380,11021278,11021366,11023034,11025536,11027914,11031016,11032664,11032864,11033381,11034124,11034407,11034607,11034831,11035558,11036067,11038914,11038926,11039510,11040046,11041479,11042580,11044729,11046547,11049819,11051185,11053247,11058766
TXN_DESC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AGODA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AGODA.COM,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AIR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AIRASIA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AIRBNB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ALI*www.taobao.com,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7,0,0,67,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ALIPAY*TAOBAO.COM,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AMAZON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AMAZON.CO.JP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# 用來方便印出使用者資料
def print_txn(n):
    print(df.iloc[n].name)

In [14]:
for i in range(5):
  print_txn(i)
  print()

AGODA

AGODA.COM

AIR

AIRASIA

AIRBNB



In [15]:
# sklearn NN
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='correlation', algorithm='brute')
model_knn.fit(df)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='correlation',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
def find_related(name):
    distances, indices = model_knn.kneighbors(df.loc[[name]], n_neighbors=6)
    for i, d in zip(indices[0][1:], distances[0][1:]):
        print(df.iloc[i].name, d)

In [17]:
find_related('台中新光影城')

新光影城股份有限公司 0.7630718751361765
新光三越百貨台中中港店 0.8151473615981774
漢村股份有限公司河南路加油站 0.8176116431461109
誠品生活股份有限公司松山分公 0.8718620606535407
新光三越名品 0.8767467638328584


In [18]:
find_related('友邦人壽')

富邦壽 0.8381436771175612
遠雄人壽保險事業股份 0.8469434037124606
全球人壽 0.8510095912540674
南桃園及／或台灣寬頻 0.8831270522000412
Xsolla 0.8918575753051762


In [19]:
find_related('AGODA')

大潤發員林店 0.8685101345489259
AGODA.COM 0.878586978279287
台亞泰安南下站 0.8959176701702065
澎坊股份有限公司三號港分公司 0.9160641260764665
樂聲影城 0.9202845378552847


In [20]:
find_related('一卡通自動加值')

統聯中轉站 0.8786249232636991
震旦通訊 0.9391964604838702
代繳遠傳帳單 0.9475749670169045
寶雅生活館台南海佃門市 0.9490445045799226
國賓影城 0.9494158400088475


In [21]:
find_related('中友百貨公司')

中友百貨股份有限公司 0.7391644363048884
中友百貨宮廷餐廳 0.7592232470301343
Ｈ＆Ｍ台中中友店 0.8487060299229123
兆豐產物保險股份有限公司 0.8538372870809443
台灣優衣庫有限公司台中崇德路 0.9067876844284717
